<a href="https://colab.research.google.com/github/jjrico/Tensor/blob/master/Tensor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensor.py
### Last edited by Jeremy Rico
### timestamp: 10/20/2020 18:23

This program creates a class Tensor that will create a tensor (n-dimensional
matrix) based on the Data and Shape input given by the user.

Import libraries and mount to Drive

In [1]:
import numpy as np # used for answer checking and testing only
import tensorflow as tf # used to import MNIST

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### Helper functions

In [2]:
# helper function to calculate the product of all elements in a list
def prod(list):
    product = 1
    for i in list:
        product = product * i
    return product

def printMat(mat):
    for row in mat:
        print(row)

In [3]:
# helper function performs dot product of two matrices by transposing weight
#   matrix and MAC-ing each row in F with each row in W_T
#   WORKS ONLY FOR 2D MATRICES
def dot(F, W):
    if (len(F[0]) != len(W)): # check that the # of cols in feautre matrix are
                              # equal to # of rows in weight matrix
        print("Error: Matrix sizes are not compatible")
        return []
    else:
        # first, transpose weights matrix
        W_T = [[W[j][i] for j in range(len(W))] for i in range(len(W[0]))]
        # now, perform multiplication
        ret = []
        # MAC each row in F with each row in W_T
        for i in range(len(F)):
            temp = []
            for j in range(len(W_T)):
                temp.append(sum(x * y for x, y in zip(F[i], W_T[j])))
            ret.append(temp)
        return ret

In [4]:
# accuracy function
def accuracy(predictions, labels):
  num_correct = 0.0
  for i in range(len(predictions)):
    if (np.argmax(predictions[i]) == labels[i]):
      num_correct += 1.0
  return num_correct/len(predictions)

### Tensor Class

In [5]:
# BEGIN TENSOR CLASS ===========================================================
class Tensor:
    # initializer
    def __init__(self, data, shape):
        self.data = data    # contains data (int, float, string, etc.)
        self.shape = shape  # shape in which to initialize the tensor
        self.tensor = []    # empty list to hold tensor
        self.create_Tensor()    # main function

    # Helper function called by constructor to shape the data.
    def create_Tensor(self):
      if (type(self.shape[0]) == list):
        for i in range(self.shape[0]):
          temp = []
          for j in range(self.shape[1]):
            temp.append(self.data[i][j])
          self.tensor.append(temp)
      else: 
        for i in self.data:
          self.tensor.append(i)
      
    def print(self):
        print(self.tensor)
# END TENSOR CLASS ============================================================

### Dense Class

In [6]:
# BEGIN DENSE CLASS ===========================================================
class Dense:
    #initializer
    def __init__(self, input, weight, bias):
        self.input = input.tensor
        self.weight = weight.tensor
        self.bias = bias.tensor
        self.layer = []

    # creates a dense layer from the three member variable lists
    def run(self):
        # find dot product of feature and weight matrix
        dot_prod = dot(self.input, self.weight) # call helper function (line 29)
        # add bias to each row
        for i in dot_prod:
            self.layer.append([sum(row) for row in zip(i, self.bias)])
        return self.layer
    
    def reLU(self):
      for i in range(len(self.layer)):
        for j in range(len(self.layer[0])):
          self.layer[i][j] = max(0.0, self.layer[i][j])
      return self.layer



# END DENSE CLASS ==============================================================

## Begin Main

### Load MNIST Data

In [7]:
# Load dataset
mnist = tf.keras.datasets.mnist

# Split into train and test 
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Regularize
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = np.reshape(x_train, (x_train.shape[0], 784))
x_test = np.reshape(x_test, (x_test.shape[0], 784))

# Add a channels dimension
#x_train = x_train[..., tf.newaxis].astype("float32")
#x_test = x_test[..., tf.newaxis].astype("float32")

### Convert MNIST to Tensor class

In [8]:
batch = 10000
x_train = Tensor(x_train[0:batch], x_train[0:batch].shape)
x_test = Tensor(x_test[0:batch], x_test[0:batch].shape)
#y_train = Tensor(y_train[0:32], y_train[0:32].shape)
#y_test = Tensor(y_test[0:32], y_test[0:32].shape)

### Load weights from Drive

In [9]:
# define data and shape to create tensors

w0_from_drive = np.load(F"/content/gdrive/My Drive/w0.npy")
b0_from_drive = np.load(F"/content/gdrive/My Drive/b0.npy")
w1_from_drive = np.load(F"/content/gdrive/My Drive/w1.npy")
b1_from_drive = np.load(F"/content/gdrive/My Drive/b1.npy")

### Convert to Tensor class

In [10]:
w0 = Tensor(w0_from_drive, w0_from_drive.shape)
b0 = Tensor(b0_from_drive, b0_from_drive.shape)
w1 = Tensor(w1_from_drive, w1_from_drive.shape)
b1 = Tensor(b1_from_drive, b1_from_drive.shape)
print(w0.shape)
print(b0.shape)
print(w1.shape)
print(b1.shape)

(784, 128)
(128,)
(128, 10)
(10,)


In [11]:
# Dense layer 1
dense0 = Dense(x_test, w0, b0)
axon0 = dense0.run()
layer1 = dense0.reLU()

#convert activation layer to tensor
layer1 = Tensor(layer1, [len(layer1), len(layer1[0])])

# Dense layer 2
dense1 = Dense(layer1, w1, b1)
axon1 = dense1.run()
layer2 = dense1.reLU() # outputs one-hot vector for digits 0-9

acc = accuracy(layer2, y_test[0:batch])
print("Model accuracy: %f%%" % (acc*100))



Model accuracy: 97.690000%
